In [14]:
import numpy as np
import pandas as pd
import sqlite3

### orders.csv

In [3]:
orders = pd.read_csv('orders.csv')

In [4]:
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [6]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB


### users.json

In [7]:
users = pd.read_json('users.json')

In [9]:
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [10]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB


### restaurants.sql

In [126]:
# Create / Open database with link
conn = sqlite3.connect('restaurants.db') 

# Create cursor to execute cmds and access results
cursor = conn.cursor()

# Open sql file in read mode
with open('restaurants.sql','r',encoding='utf-8') as f:
    sql_script = f.read()

cursor.executescript(sql_script) # execute all stmts

# Save changes
conn.commit()

In [29]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,restaurants


In [30]:
restaurants = pd.read_sql('SELECT * FROM restaurants', conn)

In [31]:
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [32]:
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant_id    500 non-null    int64  
 1   restaurant_name  500 non-null    object 
 2   cuisine          500 non-null    object 
 3   rating           500 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 15.8+ KB


### Merge Datasets

In [39]:
orders['order_date'] = pd.to_datetime(orders['order_date'], format='%d-%m-%Y')

In [40]:
orders['order_date'].dtype

dtype('<M8[ns]')

In [45]:
df = orders.merge(
    users,
    on = 'user_id',
    how = 'left'
)

In [46]:
df2 = df.merge(
    restaurants,
    on = 'restaurant_id',
    how = 'left'
)

In [47]:
df2.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [54]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   order_id           10000 non-null  int64         
 1   user_id            10000 non-null  int64         
 2   restaurant_id      10000 non-null  int64         
 3   order_date         10000 non-null  datetime64[ns]
 4   total_amount       10000 non-null  float64       
 5   restaurant_name_x  10000 non-null  object        
 6   name               10000 non-null  object        
 7   city               10000 non-null  object        
 8   membership         10000 non-null  object        
 9   restaurant_name_y  10000 non-null  object        
 10  cuisine            10000 non-null  object        
 11  rating             10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 937.6+ KB


In [55]:
df2.isna().sum()

order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
restaurant_name_x    0
name                 0
city                 0
membership           0
restaurant_name_y    0
cuisine              0
rating               0
dtype: int64

In [56]:
df2.duplicated().sum()

0

In [64]:
df2.rename(columns={
    'restaurant_name_x': 'order_restaurant_name',
    'restaurant_name_y': 'master_restaurant_name'
}, inplace=True)

In [65]:
df2.to_csv('final_food_delivery_dataset.csv', index=False)

### Questions

In [66]:
final = pd.read_csv('final_food_delivery_dataset.csv')

In [67]:
final.head()

,order_id,user_id,restaurant_id,order_date,total_amount,order_restaurant_name,name,city,membership,master_restaurant_name,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [68]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   order_id                10000 non-null  int64  
 1   user_id                 10000 non-null  int64  
 2   restaurant_id           10000 non-null  int64  
 3   order_date              10000 non-null  object 
 4   total_amount            10000 non-null  float64
 5   order_restaurant_name   10000 non-null  object 
 6   name                    10000 non-null  object 
 7   city                    10000 non-null  object 
 8   membership              10000 non-null  object 
 9   master_restaurant_name  10000 non-null  object 
 10  cuisine                 10000 non-null  object 
 11  rating                  10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [62]:
final['order_date'] = pd.to_datetime(final['order_date']).dt.date

In [63]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [71]:
# Which city has the highest total revenue (total_amount) from Gold members? 

a1 = (
    final[final['membership']=='Gold']
    .groupby('city')['total_amount']
    .sum().sort_values(ascending=False)
)

a1

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [72]:
# Which cuisine has the highest average order value across all orders?

a2 = (
    final.groupby('cuisine')['total_amount']
    .mean().sort_values(ascending=False)
)

a2

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [75]:
# How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

a__3 = (
    final.groupby('user_id')['total_amount']
    .sum().reset_index()
)

a3 = a__3[a__3['total_amount']>1000].shape[0]

a3

2544

In [124]:
# Which restaurant rating range generated the highest total revenue?

bins = [0, 2.9, 3.5, 4, 4.5, 5]
labels = ['0 - 2.9', '3.0 – 3.5', '3.6 - 4.0', '4.1 - 4.5', '4.6 - 5.0']

final['rating_range'] = pd.cut(
    final['rating'],
    bins=bins,
    labels=labels,
    include_lowest=True
)


a4 = (
    final
        .groupby('rating_range', observed=False)['total_amount']
        .sum().sort_values(ascending=False)
)

a4

rating_range
4.6 - 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 - 4.5    1960326.26
3.6 - 4.0    1717494.41
0 - 2.9            0.00
Name: total_amount, dtype: float64

In [84]:
# Among Gold members, which city has the highest average order value?

a5 = (
    final[final['membership']=='Gold']
    .groupby('city')['total_amount']
    .mean().sort_values(ascending=False)
)

a5

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [86]:
# Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

# first, group by cuisine
a6 = final.groupby('cuisine').agg(
    num_restaurants=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
).reset_index()

#  second, sort for low number of restaurants then for high revenue
a6 = a6.sort_values(
    by=['num_restaurants', 'total_revenue'],
    ascending=[True, False]
)

a6

,cuisine,num_restaurants,total_revenue
0,Chinese,120,1930504.65
2,Italian,126,2024203.80
1,Indian,126,1971412.58
3,Mexican,128,2085503.09


In [89]:
# What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

total_orders = final.shape[0]
gold_orders = final[final['membership']=='Gold'].shape[0]

a7 = round((gold_orders/total_orders)*100) # gold percent
a7

50

In [96]:
# Which restaurant has the highest average order value but less than 20 total orders?

# Group by Restaurant
a8 = final.groupby('order_restaurant_name').agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
).reset_index()

# Filter for restaurants with < 20 orders
small_rest = a8[a8['total_orders'] < 20]

# Get restaurants with highest avg order value
top_rest = small_rest.sort_values(
    by='avg_order_value', ascending=False)

# Check from options in question
a8_fin = top_rest[top_rest['order_restaurant_name'].isin(['Grand Cafe Punjabi',
                                               'Grand Restaurant South Indian',
                                               'Ruchi Mess Multicuisine',
                                               'Ruchi Foods Chinese'])] 

a8_fin

,order_restaurant_name,total_orders,avg_order_value
254,Ruchi Foods Chinese,19,686.603158


In [103]:
# Which combination contributes the highest revenue?

a9 = (
    final
        .groupby(['membership', 'cuisine'])['total_amount']
        .sum()
        .reset_index()
        .sort_values(by='total_amount', ascending=False)
)

a9

,membership,cuisine,total_amount
7,Regular,Mexican,1072943.30
6,Regular,Italian,1018424.75
3,Gold,Mexican,1012559.79
2,Gold,Italian,1005779.05
5,Regular,Indian,992100.27
1,Gold,Indian,979312.31
0,Gold,Chinese,977713.74
4,Regular,Chinese,952790.91


In [104]:
# During which quarter of the year is the total revenue highest?

final['order_date'] = pd.to_datetime(final['order_date'])

# get quarter
final['quarter'] = final['order_date'].dt.quarter

# group by quarter and revenue sum
a10 = (
    final.groupby('quarter')['total_amount']
    .sum()
    .sort_values(ascending=False)
)

a10

quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [110]:
# How many total orders were placed by users with Gold membership?

a11 = final[final['membership']=='Gold'].shape[0]
a11

4987

In [112]:
# What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

a12 = round(final[final['city']=='Hyderabad']['total_amount'].sum()) 

a12

1889367

In [114]:
# How many distinct users placed at least one order?

a13 = final['user_id'].nunique()

a13

2883

In [117]:
# What is the average order value (rounded to 2 decimals) for Gold members?

a14 = round(final[final['membership']=='Gold']['total_amount'].mean(), 2)

a14

797.15

In [118]:
# How many orders were placed for restaurants with rating ≥ 4.5?

a15 =  final[final['rating']>=4.5].shape[0]

a15

3374

In [120]:
# How many orders were placed in the top revenue city among Gold members only?

# filter gold members
filtered = final[final['membership']=='Gold']

# get top revenue city
top_city = filtered.groupby('city')['total_amount'].sum().idxmax()
print(top_city)

# count orders in that city
a16 = filtered[filtered['city']==top_city].shape[0]
a16

Chennai


1337

In [ ]:
# The column used to join orders.csv and users.json is __________.

# Ans: user_id

In [ ]:
# The dataset containing cuisine and rating information is stored in __________ format.

# Ans: sql

In [ ]:
# The total number of rows in the final merged dataset is __________.

# Ans: 10000

In [ ]:
# If a user has no matching record in users.json, the merged values will be __________.

# Ans: NaN

In [ ]:
# The Pandas function used to combine datasets based on a key is __________.

# Ans: merge()

In [ ]:
# The column membership in the final dataset originates from the __________ file.  

# Ans: users.json

In [ ]:
# The join key used to combine orders data with restaurant details is __________.  

# Ans: restaurant_id

In [ ]:
# The column that helps identify the type of food served by a restaurant is __________.  

# Ans: cuisine

In [ ]:
# If a user places multiple orders, their personal details appear __________ times in the final merged dataset.  

# Ans: multiple